<a href="https://colab.research.google.com/github/sanadv/Cubixel-City-Exploration-Framework/blob/main/Cubixel_City_Agents_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_fscore_support
from tensorflow.keras.callbacks import Callback

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Cubixel Transformation
def cubixel_transform(image):
    cubixels = np.zeros((image.shape[0], image.shape[1], 3))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            R, G, B = image[i, j]
            cubixels[i, j] = [R, G, B]  # Cubixel dimensions (R,G,B)
    return cubixels

# VoV Computation
def compute_vov(cubixels):
    VoV = np.zeros((cubixels.shape[0], cubixels.shape[1]))
    for i in range(1, cubixels.shape[0] - 1):
        for j in range(1, cubixels.shape[1] - 1):
            current_vol = np.prod(cubixels[i, j])
            neighbors = [
                np.prod(cubixels[i - 1, j]), np.prod(cubixels[i + 1, j]),
                np.prod(cubixels[i, j - 1]), np.prod(cubixels[i, j + 1])
            ]
            VoV[i, j] = np.sum(np.abs(np.array(neighbors) - current_vol))
    return VoV

# Agent Class for Exploring VoV Map
class Agent:
    def __init__(self, grid, start_pos=(0, 0)):
        self.grid = grid  # VoV map
        self.position = start_pos
        self.explored = set()

    def move(self):
        x, y = self.position
        neighbors = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
        valid_neighbors = [(i, j) for i, j in neighbors if 0 <= i < self.grid.shape[0] and 0 <= j < self.grid.shape[1]]

        # Move to the neighbor with the highest VoV
        best_move = max(valid_neighbors, key=lambda pos: self.grid[pos])
        self.position = best_move
        self.explored.add(best_move)
        return best_move

    def explore(self, steps=100):
        path = []
        for _ in range(steps):
            next_pos = self.move()
            path.append(next_pos)
        return path

# Extract Features using Cubixel and VoV + Agent
def extract_features(image):
    cubixels = cubixel_transform(image)
    vov_map = compute_vov(cubixels)

    # Initialize agent and explore
    agent = Agent(vov_map)
    path = agent.explore(steps=50)

    # Create a feature map combining local VoV and agent exploration
    feature_map = np.zeros_like(vov_map)
    for x, y in path:
        feature_map[x, y] = vov_map[x, y]  # Mark agent path

    # Combine VoV map and agent exploration map as hybrid features
    hybrid_features = np.stack([vov_map, feature_map], axis=-1)
    return hybrid_features

# Apply feature extraction to the entire dataset
def transform_dataset(dataset):
    transformed_dataset = []
    for image in dataset:
        features = extract_features(image)
        transformed_dataset.append(features)
    return np.array(transformed_dataset)

# Transform training and testing datasets
x_train_transformed = transform_dataset(x_train)
x_test_transformed = transform_dataset(x_test)

# Define the CNN model
def build_cnn_model():
    model = models.Sequential()

    # Input shape: (32, 32, 2) because we have 2 feature maps (VoV and agent-based map)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 2)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  # 10 classes for CIFAR-10

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Custom callback to compute additional metrics after training
class MetricsCallback(Callback):
    def on_train_end(self, logs=None):
        # Training predictions
        y_train_pred = np.argmax(self.model.predict(x_train_transformed), axis=-1)
        y_train_true = np.argmax(y_train, axis=-1)

        # Testing predictions
        y_test_pred = np.argmax(self.model.predict(x_test_transformed), axis=-1)
        y_test_true = np.argmax(y_test, axis=-1)

        # Training metrics
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(y_train_true, y_train_pred, average='macro')
        train_auc = roc_auc_score(y_train, self.model.predict(x_train_transformed), multi_class='ovr')
        train_loss, train_acc = self.model.evaluate(x_train_transformed, y_train, verbose=0)

        # Testing metrics
        test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(y_test_true, y_test_pred, average='macro')
        test_auc = roc_auc_score(y_test, self.model.predict(x_test_transformed), multi_class='ovr')
        test_loss, test_acc = self.model.evaluate(x_test_transformed, y_test, verbose=0)

        # Print the results
        print("\nTraining Metrics:")
        print(f"Loss: {train_loss:.4f}, Accuracy: {train_acc * 100:.2f}%")
        print(f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1-score: {train_f1:.4f}, AUC: {train_auc:.4f}")

        print("\nTesting Metrics:")
        print(f"Loss: {test_loss:.4f}, Accuracy: {test_acc * 100:.2f}%")
        print(f"Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1-score: {test_f1:.4f}, AUC: {test_auc:.4f}")

# Build the model
model = build_cnn_model()

# Train the model with custom callback
metrics_callback = MetricsCallback()
history = model.fit(x_train_transformed, y_train, epochs=50, batch_size=64, validation_data=(x_test_transformed, y_test), callbacks=[metrics_callback])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test_transformed, y_test)
print(f'\nFinal Test accuracy: {test_acc * 100:.2f}%')


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_fscore_support
from tensorflow.keras.callbacks import Callback

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape to add a single grayscale channel
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Normalize the pixel values
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Cubixel Transformation for Grayscale Images
def cubixel_transform(image):
    cubixels = np.zeros((image.shape[0], image.shape[1], 1))  # Only one channel for grayscale
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel_value = image[i, j, 0]  # Grayscale pixel value
            cubixels[i, j] = [pixel_value]  # Single cubixel dimension (Grayscale)
    return cubixels

# VoV Computation
def compute_vov(cubixels):
    VoV = np.zeros((cubixels.shape[0], cubixels.shape[1]))
    for i in range(1, cubixels.shape[0] - 1):
        for j in range(1, cubixels.shape[1] - 1):
            current_vol = np.prod(cubixels[i, j])
            neighbors = [
                np.prod(cubixels[i - 1, j]), np.prod(cubixels[i + 1, j]),
                np.prod(cubixels[i, j - 1]), np.prod(cubixels[i, j + 1])
            ]
            VoV[i, j] = np.sum(np.abs(np.array(neighbors) - current_vol))
    return VoV

# Agent Class for Exploring VoV Map
class Agent:
    def __init__(self, grid, start_pos=(0, 0)):
        self.grid = grid  # VoV map
        self.position = start_pos
        self.explored = set()

    def move(self):
        x, y = self.position
        neighbors = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
        valid_neighbors = [(i, j) for i, j in neighbors if 0 <= i < self.grid.shape[0] and 0 <= j < self.grid.shape[1]]

        # Move to the neighbor with the highest VoV
        best_move = max(valid_neighbors, key=lambda pos: self.grid[pos])
        self.position = best_move
        self.explored.add(best_move)
        return best_move

    def explore(self, steps=100):
        path = []
        for _ in range(steps):
            next_pos = self.move()
            path.append(next_pos)
        return path

# Extract Features using Cubixel and VoV + Agent
def extract_features(image):
    cubixels = cubixel_transform(image)
    vov_map = compute_vov(cubixels)

    # Initialize agent and explore
    agent = Agent(vov_map)
    path = agent.explore(steps=50)

    # Create a feature map combining local VoV and agent exploration
    feature_map = np.zeros_like(vov_map)
    for x, y in path:
        feature_map[x, y] = vov_map[x, y]  # Mark agent path

    # Combine VoV map and agent exploration map as hybrid features
    hybrid_features = np.stack([vov_map, feature_map], axis=-1)
    return hybrid_features

# Apply feature extraction to the entire dataset
def transform_dataset(dataset):
    transformed_dataset = []
    for image in dataset:
        features = extract_features(image)
        transformed_dataset.append(features)
    return np.array(transformed_dataset)

# Transform training and testing datasets
x_train_transformed = transform_dataset(x_train)
x_test_transformed = transform_dataset(x_test)

# Define the CNN model
def build_cnn_model():
    model = models.Sequential()

    # Input shape: (28, 28, 2) because we have 2 feature maps (VoV and agent-based map)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 2)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  # 10 classes for MNIST

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Custom callback to compute additional metrics after training
class MetricsCallback(Callback):
    def on_train_end(self, logs=None):
        # Training predictions
        y_train_pred = np.argmax(self.model.predict(x_train_transformed), axis=-1)
        y_train_true = np.argmax(y_train, axis=-1)

        # Testing predictions
        y_test_pred = np.argmax(self.model.predict(x_test_transformed), axis=-1)
        y_test_true = np.argmax(y_test, axis=-1)

        # Training metrics
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(y_train_true, y_train_pred, average='macro')
        train_auc = roc_auc_score(y_train, self.model.predict(x_train_transformed), multi_class='ovr')
        train_loss, train_acc = self.model.evaluate(x_train_transformed, y_train, verbose=0)

        # Testing metrics
        test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(y_test_true, y_test_pred, average='macro')
        test_auc = roc_auc_score(y_test, self.model.predict(x_test_transformed), multi_class='ovr')
        test_loss, test_acc = self.model.evaluate(x_test_transformed, y_test, verbose=0)

        # Print the results
        print("\nTraining Metrics:")
        print(f"Loss: {train_loss:.4f}, Accuracy: {train_acc * 100:.2f}%")
        print(f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1-score: {train_f1:.4f}, AUC: {train_auc:.4f}")

        print("\nTesting Metrics:")
        print(f"Loss: {test_loss:.4f}, Accuracy: {test_acc * 100:.2f}%")
        print(f"Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1-score: {test_f1:.4f}, AUC: {test_auc:.4f}")

# Build the model
model = build_cnn_model()

# Train the model with custom callback
metrics_callback = MetricsCallback()
history = model.fit(x_train_transformed, y_train, epochs=50, batch_size=64, validation_data=(x_test_transformed, y_test), callbacks=[metrics_callback])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test_transformed, y_test)
print(f'\nFinal Test accuracy: {test_acc * 100:.2f}%')
